<a href="https://colab.research.google.com/github/campeone-pixel/soccer_database/blob/master/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import files

In [11]:
lista_ligas = [
    "https://fbref.com/en/comps/12/2021-2022/2021-2022-La-Liga-Stats",
    "https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats",
    "https://fbref.com/en/comps/11/2021-2022/2021-2022-Serie-A-Stats",
    "https://fbref.com/en/comps/13/2021-2022/2021-2022-Ligue-1-Stats",
    "https://fbref.com/en/comps/20/2021-2022/2021-2022-Bundesliga-Stats",
]

years = list(range(2021, 2013, -1))
all_matches = []
log_error = []
log = []
for year in years:
    for indice, liga in enumerate(lista_ligas):
        data = requests.get(liga)
        soup = BeautifulSoup(data.text)

        try:
            standings_table = soup.select("table.stats_table")[0]
        except Exception as e:
            
            continue

        links = [l.get("href") for l in standings_table.find_all("a")]
        links = [l for l in links if "/squads/" in l]
        team_urls = [f"https://fbref.com{l}" for l in links]

        previous_season = soup.select("a.prev")[0].get("href")
        

        time.sleep(1)
        for team_url in team_urls:
            
            team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
            data = requests.get(team_url)
            time.sleep(1)
            try:
                matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
                matches = matches[
                    [
                        "Date",
                        "Time",
                        "Comp",
                        "Round",
                        "Venue",
                        "Result",
                        "GF",
                        "GA",
                        "Opponent",
                        "Poss",
                        "Formation",
                    ]
                ]
                soup = BeautifulSoup(data.text)
                links = [l.get("href") for l in soup.find_all("a")]
                time.sleep(1)
            except Exception as e:
                log_error.append(f"error:{e},{lista_ligas[indice]},{team_url}")
                continue

            try:
                links_shooting = [l for l in links if l and "all_comps/shooting/" in l]
                data_shooting = requests.get(f"https://fbref.com{links_shooting[0]}")
                shooting = pd.read_html(data_shooting.text, match="Shooting")[0]
                shooting.columns = shooting.columns.droplevel()
                time.sleep(1)
            except Exception as e:
                
                continue

            try:
                links_misc = [l for l in links if l and "all_comps/misc/" in l]
                data_misc = requests.get(f"https://fbref.com{links_misc[0]}")
                defensive = pd.read_html(data_misc.text, match="Miscellaneous Stats")[0]
                defensive.columns = defensive.columns.droplevel()
                time.sleep(1)

            except Exception as e:
                continue

            try:
                team_data = matches.merge(defensive[["Date", "TklW", "Int"]], on="Date")
            except ValueError:
                continue

            try: 
                if 'xG' in shooting.columns:
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]],
                        on="Date",
                    )
                    print('entre en el if ',team_url)
                    
                    
                else:
                    shooting["xG"] = ''
                    team_data = team_data.merge(
                    shooting[["Date", "Sh", "SoT", "Dist", "PK", "PKatt", "xG"]], on="Date"
                )
                    print('entre en el else ',team_url)
                   

            except Exception as e:
              
                continue

            team_data["Season"] = year
            team_data["Team"] = team_name
            all_matches.append(team_data)
            time.sleep(2)
           

        lista_ligas[indice] = f"https://fbref.com{previous_season}"

    all_matches[-1].to_csv('parcial.csv')
    files.download('parcial.csv')
        


entre en el if  https://fbref.com/en/squads/53a2f082/2021-2022/Real-Madrid-Stats
entre en el if  https://fbref.com/en/squads/206d90db/2021-2022/Barcelona-Stats
entre en el if  https://fbref.com/en/squads/db3b9613/2021-2022/Atletico-Madrid-Stats
entre en el if  https://fbref.com/en/squads/ad2be733/2021-2022/Sevilla-Stats
entre en el if  https://fbref.com/en/squads/fc536746/2021-2022/Real-Betis-Stats
entre en el if  https://fbref.com/en/squads/e31d1cd9/2021-2022/Real-Sociedad-Stats
entre en el if  https://fbref.com/en/squads/2a8183b3/2021-2022/Villarreal-Stats
entre en el if  https://fbref.com/en/squads/2b390eca/2021-2022/Athletic-Club-Stats
entre en el if  https://fbref.com/en/squads/dcc91a7b/2021-2022/Valencia-Stats
entre en el if  https://fbref.com/en/squads/03c57e2b/2021-2022/Osasuna-Stats
entre en el if  https://fbref.com/en/squads/f25da7fb/2021-2022/Celta-Vigo-Stats
entre en el if  https://fbref.com/en/squads/98e8af82/2021-2022/Rayo-Vallecano-Stats
entre en el if  https://fbref.com

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

entre en el if  https://fbref.com/en/squads/db3b9613/2020-2021/Atletico-Madrid-Stats
entre en el if  https://fbref.com/en/squads/53a2f082/2020-2021/Real-Madrid-Stats
entre en el if  https://fbref.com/en/squads/206d90db/2020-2021/Barcelona-Stats
entre en el if  https://fbref.com/en/squads/ad2be733/2020-2021/Sevilla-Stats
entre en el if  https://fbref.com/en/squads/e31d1cd9/2020-2021/Real-Sociedad-Stats
entre en el if  https://fbref.com/en/squads/fc536746/2020-2021/Real-Betis-Stats
entre en el if  https://fbref.com/en/squads/2a8183b3/2020-2021/Villarreal-Stats
entre en el if  https://fbref.com/en/squads/f25da7fb/2020-2021/Celta-Vigo-Stats
entre en el if  https://fbref.com/en/squads/a0435291/2020-2021/Granada-Stats
entre en el if  https://fbref.com/en/squads/2b390eca/2020-2021/Athletic-Club-Stats
entre en el if  https://fbref.com/en/squads/03c57e2b/2020-2021/Osasuna-Stats
entre en el if  https://fbref.com/en/squads/ee7c297c/2020-2021/Cadiz-Stats
entre en el if  https://fbref.com/en/squads

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

entre en el else  https://fbref.com/en/squads/206d90db/2014-2015/Barcelona-Stats
entre en el else  https://fbref.com/en/squads/53a2f082/2014-2015/Real-Madrid-Stats
entre en el else  https://fbref.com/en/squads/db3b9613/2014-2015/Atletico-Madrid-Stats
entre en el else  https://fbref.com/en/squads/dcc91a7b/2014-2015/Valencia-Stats
entre en el else  https://fbref.com/en/squads/ad2be733/2014-2015/Sevilla-Stats
entre en el else  https://fbref.com/en/squads/2a8183b3/2014-2015/Villarreal-Stats
entre en el else  https://fbref.com/en/squads/2b390eca/2014-2015/Athletic-Club-Stats
entre en el else  https://fbref.com/en/squads/f25da7fb/2014-2015/Celta-Vigo-Stats
entre en el else  https://fbref.com/en/squads/1c896955/2014-2015/Malaga-Stats
entre en el else  https://fbref.com/en/squads/a8661628/2014-2015/Espanyol-Stats
entre en el else  https://fbref.com/en/squads/98e8af82/2014-2015/Rayo-Vallecano-Stats
entre en el else  https://fbref.com/en/squads/e31d1cd9/2014-2015/Real-Sociedad-Stats
entre en el 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
match_df = pd.concat(all_matches)
match_df.columns = [c.lower() for c in match_df.columns]
match_df = match_df[['date', 'time', 'season', 'team', 'comp', 'round', 'venue', 'result', 'gf', 'ga',
       'opponent', 'poss', 'formation', 'tklw', 'int', 'sh', 'sot', 'dist',
        'xg']]
match_df.to_csv('matches.csv')
files.download('matches.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
match_df.tail()

In [ ]:
all_matches[-1]